In [1]:
# Importation des bibliothèques nécessaires
import os
import re
import json
import time
from pathlib import Path
from dotenv import load_dotenv
import google.generativeai as genai

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()
api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_gemini_api'.")

# Configurer l'API Gemini
genai.configure(api_key=api_key)

/home/maximeadmin/anaconda3/envs/mon_env_ia/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prompt

In [2]:


def create_prompt(latex_content):
    """Créer le prompt pour l'API Gemini."""
    return f"""Voici un exercice de mathématiques au format LaTeX. Analyse-le et génère un résumé synthétique de l'exercice. 
Ne réécris pas les questions, formule plutôt les compétences mises en oeuvre et sois concis. 
Si tu utilises des formules mathématiques, mets les entre $.
Réponds uniquement en format JSON structuré avec cette clé exacte:
{{
  "resume": "résumé"
}}

Voici l'exercice:
{latex_content}"""

## Extraction des données

In [3]:
def extract_uuid(content):
    """Extraire l'UUID du contenu LaTeX."""
    uuid_match = re.search(r'\\uuid\{([^}]+)\}', content)
    return uuid_match.group(1) if uuid_match else None

def extract_json_from_response(text_response, debug=False):
    """
    Extraire le contenu du résumé avec plusieurs méthodes en cas d'échec.
    """
    if debug:
        print("\n=== RÉPONSE BRUTE DE L'API ===")
        print(text_response)
        print("==============================\n")
    
    # Approche 1: Parsing JSON standard
    try:
        json_data = json.loads(text_response)
        return json_data
    except json.JSONDecodeError:
        pass
    
    # Approche 2: Extraction directe du résumé via regex
    resume_match = re.search(r'"resume"\s*:\s*"([\s\S]*?)(?:"(?=\s*})|"(?=\s*,))', text_response)
    if resume_match:
        resume_content = resume_match.group(1)
        return {"resume": resume_content}
    
    # Approche 3: Extraction depuis un bloc de code
    code_block_match = re.search(r'```(?:json)?\s*({[\s\S]*?})\s*```', text_response)
    if code_block_match:
        try:
            json_code = code_block_match.group(1)
            json_data = json.loads(json_code)
            return json_data
        except json.JSONDecodeError:
            resume_in_block = re.search(r'"resume"\s*:\s*"([\s\S]*?)(?:"(?=\s*})|"(?=\s*,))', json_code)
            if resume_in_block:
                return {"resume": resume_in_block.group(1)}
    
    # Approche 4: Recherche générique
    any_resume = re.search(r'resume["\':].*?([\w\s\d\.,;:!?\(\)\[\]\{\}\-_\+=\*\&\^%\$\#@~`<>\\\\/]+)', text_response, re.IGNORECASE)
    if any_resume:
        return {"resume": any_resume.group(1).strip()}
    
    # Dernière solution : capturer le début de la réponse
    return {"resume": "EXTRACTION MANUELLE NÉCESSAIRE: " + text_response[:300] + "..."}

In [4]:
def analyze_latex_file(file_path, output_dir="output", model_name="gemini-2.0-flash", force_reprocess=False, debug=False):
    """Analyser un fichier LaTeX et générer les métadonnées via l'API Gemini."""
    try:
        # Lire le contenu du fichier
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extraire l'UUID
        uuid = extract_uuid(content)
        if not uuid:
            print(f"UUID non trouvé dans le fichier {file_path}")
            return None
        
        # Vérifier si le fichier a déjà été traité
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, f"{uuid}.json")
        if os.path.exists(output_file) and not force_reprocess:
            print(f"Fichier {uuid} déjà traité, ignoré.")
            return {"uuid": uuid, "status": "skipped"}
        
        # Créer le prompt
        prompt = create_prompt(content)
        
        if debug:
            print("\n=== PROMPT ENVOYÉ À L'API ===")
            print(prompt)
            print("============================\n")
        
        # Appeler l'API Gemini
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            text_response = response.text
        except Exception as e:
            print(f"ERREUR lors de l'appel à l'API Gemini: {e}")
            return None
        
        # Extraire les données de la réponse
        try:
            json_data = extract_json_from_response(text_response, debug)
            
            metadata = {
                "resume": json_data.get("resume", "")
            }
            
            # Création du résultat et écriture dans le fichier
            result = {uuid: metadata}
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            
            return result
        except Exception as e:
            print(f"Erreur lors de l'extraction des métadonnées pour {uuid}: {e}")
            if debug:
                print(f"Réponse brute: {text_response}")
            return None
    except Exception as e:
        print(f"Erreur lors de l'analyse du fichier {file_path}: {e}")
        return None

In [7]:
def process_latex_files(source_dir="src/latex/amscc", output_dir="output", model_name="gemini-2.0-flash", 
                       max_files=None, api_delay=5.0, force_reprocess=False, debug=False):
    """Traiter tous les fichiers LaTeX dans le répertoire source."""
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtenir tous les fichiers .tex dans le répertoire source
    files = [str(f) for f in Path(source_dir).glob('**/*.tex')]
    
    # Limiter le nombre de fichiers si demandé
    if max_files:
        files = files[:max_files]
        print(f"Mode test: limité à {max_files} fichiers")
    
    print(f"Traitement de {len(files)} fichiers LaTeX...")
    
    # Traiter chaque fichier
    results = {}
    success_count = 0
    skipped_count = 0
    failed_count = 0
    
    for i, file in enumerate(files):
        print(f"\n[{i+1}/{len(files)}] Analyse de {os.path.basename(file)}...")
        result = analyze_latex_file(file, output_dir, model_name, force_reprocess, debug)
        
        if result is None:
            failed_count += 1
            # Pause uniquement après traitement réel (échec) de l'API
            if i < len(files) - 1:
                print(f"Pause de {api_delay} secondes...")
                time.sleep(api_delay)
        elif "status" in result and result["status"] == "skipped":
            skipped_count += 1
            # Pas de pause pour les fichiers ignorés
        else:
            results.update(result)
            success_count += 1
            # Pause après traitement réel (succès) de l'API
            if i < len(files) - 1:
                print(f"Pause de {api_delay} secondes...")
                time.sleep(api_delay)
    
    # Écrire tous les résultats dans un seul fichier
    with open(os.path.join(output_dir, 'all_results.json'), 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"Traitement terminé. {success_count} fichiers analysés avec succès, {skipped_count} ignorés, {failed_count} échoués.")
    return results

In [8]:
# Exemple d'utilisation
results = process_latex_files(
    source_dir="../src/latex/amscc",
    output_dir="../metadata/amscc-resume",
    force_reprocess=False,  # Ne pas retraiter les fichiers déjà analysés
    api_delay=5,            # Pause de 5 secondes entre les appels API
    debug=False             # Activer pour plus de détails sur le traitement
)

Traitement de 460 fichiers LaTeX...

[1/460] Analyse de YBwt.tex...
Fichier YBwt déjà traité, ignoré.

[2/460] Analyse de trhY.tex...
Fichier trhY déjà traité, ignoré.

[3/460] Analyse de aWAS.tex...
Fichier aWAS déjà traité, ignoré.

[4/460] Analyse de DEZs.tex...
Fichier DEZs déjà traité, ignoré.

[5/460] Analyse de 5cbY.tex...
Fichier 5cbY déjà traité, ignoré.

[6/460] Analyse de ouYw.tex...
Fichier ouYw déjà traité, ignoré.

[7/460] Analyse de 6NIK.tex...
Fichier 6NIK déjà traité, ignoré.

[8/460] Analyse de PPhD.tex...
Fichier PPhD déjà traité, ignoré.

[9/460] Analyse de 0oHk.tex...
Fichier 0oHk déjà traité, ignoré.

[10/460] Analyse de hWFx.tex...
Fichier hWFx déjà traité, ignoré.

[11/460] Analyse de Uijb.tex...
Fichier Uijb déjà traité, ignoré.

[12/460] Analyse de 7aqI.tex...
Fichier 7aqI déjà traité, ignoré.

[13/460] Analyse de cEVS.tex...
Fichier cEVS déjà traité, ignoré.

[14/460] Analyse de tNNT.tex...
Fichier tNNT déjà traité, ignoré.

[15/460] Analyse de gsn9.tex...
Fi